# ALeRCE classes

https://github.com/ZwickyTransientFacility/ztf-avro-alert

1. **AGN:** Active Galactic Nuclei
1. **Blazar:** Blazar
1. **CV/Nova:** Cataclysmic Variable Star/Nova
1. **Ceph:** Cepheid Variable Star
1. **DSCT:** Delta Scuti Star
1. **EA:** Eclipsing Algol
1. **EB/EW:** Eclipsing Binaries/Eclipsing W Ursa Majoris
1. **LPV:** Long Period Variable
1. **Periodic-Other:** Periodic-Other
1. **QSO:** Quasi-Stellar Object
1. **RRL:** RRLyrae Variable Star
1. **RSCVn:** RS Canum Venaticorum
1. **SLSN:** Super Luminous Supernova
1. **SNII:** Supernova II
1. **SNIIb:** Supernova IIb
1. **SNIIn:** Supernova IIn
1. **SNIa:** Supernova Ia
1. **SNIbc:** Supernova Ibc
1. **TDE:** Tidal disruption event (to remove)
1. **YSO:** Young Stellar Object
1. **ZZ:** ZZ Ceti Stars (to remove)

In [1]:
import sys
sys.path.append('../../')

In [2]:
%load_ext autoreload
%autoreload 2
from vstars.alerce_utils import process_df_labels, process_df_detections, keep_only_valid_objs
import numpy as np
import pandas as pd

load_rootdir = '../../../../tesis/surveys_data'
survey_name = 'alerceZTFv7.1'
uses_corr = False # uses_corr=False only can be used with SNe objects
df_index_names = {
    'oid':'oid', # object id
    'oid_det':'objectId', # object id
    'label':'classALeRCE', # object class name
    'ra':'ra',
    'dec':'dec',
    'band':'fid', # band
    'obs_day':'mjd', # days
    'obs':'magpsf_corr' if uses_corr else 'magpsf', # observations
    'obs_error':'sigmapsf_corr' if uses_corr else 'sigmapsf', # observation errors
}
detections_cols = ['objectId', 'fid', 'mjd', df_index_names['obs'], df_index_names['obs_error']]

### load files and processing
detections_df = pd.read_parquet(f'{load_rootdir}/{survey_name}/detections_with_xmatch')
detections_df, det_objs = process_df_detections(detections_df, df_index_names['oid_det'], df_index_names['oid'], detections_cols, uses_corr=uses_corr)
print(f'detections_df - columns: {list(detections_df.columns)} - id: {detections_df.index.name}')

labels_df = pd.read_csv(f'{load_rootdir}/{survey_name}/dfcrossmatches_prioritized_v7.0.1.csv')
labels_df, label_objs = process_df_labels(labels_df, df_index_names['oid'], det_objs)
print(f'labels - columns: {list(labels_df.columns)} - id: {labels_df.index.name}')

### filter
valid_objs = list(set(det_objs) & set(label_objs))
labels_df = keep_only_valid_objs(labels_df, valid_objs)
detections_df = keep_only_valid_objs(detections_df, valid_objs)

### print info
classes = set(labels_df[df_index_names['label']].values)
print('classes:', classes)

../../vstars/alerce_utils.py:80: UserWarning: only use uses_corr=False with SNe objects
  warnings.warn('only use uses_corr=False with SNe objects')


detections_df - columns: ['fid', 'mjd', 'magpsf', 'sigmapsf'] - id: oid
labels - columns: ['classALeRCE', 'ra', 'dec', 'period', 'source', 'id_source', 'class_source', 'separation_arcsec'] - id: oid
classes: {'EB/EW', 'Ceph', 'NLAGN', 'SNIIb', 'DSCT', 'AGN', 'LPV', 'RRL', 'CV/Nova', 'QSO', 'EA', 'Blazar', 'YSO', 'SNIIn', 'NLQSO', 'Periodic-Other', 'RSCVn', 'SLSN', 'SNIa', 'ZZ', 'TDE', 'SNII', 'SNIbc'}


In [3]:
%load_ext autoreload
%autoreload 2
from vstars.level_bars import LevelBar

classes, counts = np.unique(labels_df[df_index_names['label']].values, return_counts=True)
population_cdict = {c:counts[kc] for kc,c in enumerate(classes)}
print(LevelBar(population_cdict, ncols=60))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
|█▌                                                        | AGN - 3,177/122,825 (2.59%)
|▍                                                         | Blazar - 1,019/122,825 (0.83%)
|▍                                                         | CV/Nova - 981/122,825 (0.80%)
|▎                                                         | Ceph - 716/122,825 (0.58%)
|▍                                                         | DSCT - 946/122,825 (0.77%)
|███▍                                                      | EA - 7,304/122,825 (5.95%)
|██████████████████▌                                       | EB/EW - 39,353/122,825 (32.04%)
|██████▋                                                   | LPV - 14,207/122,825 (11.57%)
|                                                          | NLAGN - 6/122,825 (0.00%)
|                                                          | NLQSO - 77/122,825 (0.06%)
|▏               

In [4]:
detections_df.info()
detections_df[-20:]

<class 'pandas.core.frame.DataFrame'>
Index: 6210462 entries, ZTF17aaaemke to ZTF20abefeou
Data columns (total 4 columns):
fid         int64
mjd         float64
magpsf      float32
sigmapsf    float32
dtypes: float32(2), float64(1), int64(1)
memory usage: 189.5+ MB


,fid,mjd,magpsf,sigmapsf
oid,,,,
ZTF20abcxmfu,2,59063.378160,19.111362,0.120889
ZTF20abcxmfu,2,59087.373218,18.980789,0.069250
ZTF20abcxmfu,2,59026.475370,18.813822,0.084885
ZTF20abcxmfu,2,59078.334687,19.052214,0.074238
ZTF20abcxmfu,2,59063.372917,18.973995,0.132204
ZTF20abcxmfu,2,59075.331447,19.068327,0.082464
ZTF20abcxmfu,2,59036.376898,18.869030,0.117595
ZTF20abcxmfu,2,59072.331921,18.976032,0.091529
ZTF20abcxmfu,2,59107.275880,19.158592,0.144395


In [5]:
labels_df.info()
labels_df[:20]

<class 'pandas.core.frame.DataFrame'>
Index: 122825 entries, ZTF19aapcxhy to ZTF18abgqxlw
Data columns (total 8 columns):
classALeRCE          122825 non-null object
ra                   122825 non-null float64
dec                  122825 non-null float64
period               77759 non-null object
source               122825 non-null object
id_source            122825 non-null object
class_source         122825 non-null object
separation_arcsec    122825 non-null float64
dtypes: float64(3), object(5)
memory usage: 8.4+ MB


,classALeRCE,ra,dec,period,source,id_source,class_source,separation_arcsec
oid,,,,,,,,
ZTF19aapcxhy,AGN,154.202129,18.723076,NaN,Oh2015,5.877420127343739e+17,AGN_galaxy_dominated,0.227455
ZTF18abtyspw,AGN,25.660298,0.087434,NaN,Oh2015,5.880155092805878e+17,AGN_galaxy_dominated,0.141792
ZTF18abwtbad,AGN,51.846346,0.739559,NaN,Oh2015,5.877315136939624e+17,AGN_galaxy_dominated,0.084636
ZTF18abtmwvo,AGN,46.074050,0.474212,NaN,Oh2015,5.880155098263717e+17,AGN_galaxy_dominated,0.471111
ZTF18acvgdfy,AGN,134.407409,5.472596,NaN,Oh2015,5.877327033915148e+17,AGN_galaxy_dominated,0.087190
ZTF18aadyxlg,AGN,125.577004,33.091120,NaN,Oh2015,5.880133827239608e+17,AGN_galaxy_dominated,0.073570
ZTF19aapuscr,AGN,199.102327,-2.090396,NaN,Oh2015,5.87724649802236e+17,AGN_galaxy_dominated,0.038667
ZTF19aanxuxz,AGN,212.636953,-2.822530,NaN,Oh2015,5.877297769066006e+17,AGN_galaxy_dominated,0.181073
ZTF19aaaejth,AGN,141.748427,4.036104,NaN,Oh2015,5.880103596064113e+17,AGN_galaxy_dominated,0.097429


In [6]:
%load_ext autoreload
%autoreload 2
from vstars.alerce_utils import get_valid_classes_objs, keep_only_valid_objs
    
target_classes = [
    #'EA', # Eclipsing Binaries
    'EB/EW', # Eclipsing Binaries
    'Ceph', # Cefeidas
    'RRL', # RR Lyrae
    'DSCT', # Delta Scuti
    'LPV', # Long Period Variables
]
valid_objs = get_valid_classes_objs(labels_df, df_index_names, target_classes)
print(valid_objs[:10])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['ZTF18abwwdsc', 'ZTF18aaakigd', 'ZTF18aaavkyj', 'ZTF18abwwdxw', 'ZTF19aaocniv', 'ZTF17aacemqz', 'ZTF18aaiyfjx', 'ZTF19aaczymt', 'ZTF18abvpirg', 'ZTF18abccnft']


In [7]:
new_detections_df = keep_only_valid_objs(detections_df, valid_objs)
print(new_detections_df.info())
new_detections_df[:50]

<class 'pandas.core.frame.DataFrame'>
Index: 4656626 entries, ZTF17aaaemke to ZTF20abefeou
Data columns (total 4 columns):
fid         int64
mjd         float64
magpsf      float32
sigmapsf    float32
dtypes: float32(2), float64(1), int64(1)
memory usage: 142.1+ MB
None


,fid,mjd,magpsf,sigmapsf
oid,,,,
ZTF17aaaemke,1,58679.445093,18.297556,0.107698
ZTF17aaaemke,1,58667.458588,18.762489,0.176661
ZTF17aaaemke,1,58811.170556,19.481199,0.276603
ZTF17aaaemke,1,58863.108241,19.077507,0.134806
ZTF17aaaemke,1,58820.147292,19.210100,0.281415
ZTF17aaaemke,1,58673.479028,18.959806,0.165157
ZTF17aaaemke,1,58431.211875,18.543581,0.138088
ZTF17aaaemke,2,58757.320671,18.332430,0.177623
ZTF17aaaemke,2,58667.463843,17.781448,0.086865


In [8]:
new_labels_df = keep_only_valid_objs(labels_df, valid_objs)
print(new_labels_df.info())
new_labels_df[:50]

<class 'pandas.core.frame.DataFrame'>
Index: 90481 entries, ZTF18abwwdsc to ZTF18abgqxlw
Data columns (total 8 columns):
classALeRCE          90481 non-null object
ra                   90481 non-null float64
dec                  90481 non-null float64
period               67464 non-null object
source               90481 non-null object
id_source            90481 non-null object
class_source         90481 non-null object
separation_arcsec    90481 non-null float64
dtypes: float64(3), object(5)
memory usage: 6.2+ MB
None


,classALeRCE,ra,dec,period,source,id_source,class_source,separation_arcsec
oid,,,,,,,,
ZTF18abwwdsc,Ceph,326.755845,-8.060673,2.18895,CRTSnorth,1007116010622.0,Cep-II,0.676851
ZTF18aaakigd,Ceph,65.440016,34.069809,2.11360,CRTSnorth,1135020009815.0,ACEP,0.390734
ZTF18aaavkyj,Ceph,192.885044,24.122642,1.09572,CRTSnorth,1123064031941.0,ACEP,0.291110
ZTF18abwwdxw,Ceph,324.865973,-17.296145,1.11714,CRTSnorth,1018112055304.0,ACEP,1.881608
ZTF19aaocniv,Ceph,248.580619,-16.015739,1.3049582,CRTSnorth,1015086048064.0,Cep-II,0.405458
ZTF17aacemqz,Ceph,117.977671,27.561363,16.2488,CRTSnorth,1126038065052.0,Cep-II,1.092943
ZTF18aaiyfjx,Ceph,245.464370,38.909590,1.04986,CRTSnorth,1138071060852.0,Cep-II,0.610218
ZTF19aaczymt,Ceph,228.810756,-14.901231,4.3680244,CRTSnorth,1015079060794.0,Cep-II,0.422664
ZTF18abvpirg,Ceph,351.955300,-9.388094,119.947,CRTSnorth,1009125035570.0,Cep-II,0.266861


In [9]:
print(new_labels_df.loc['ZTF18aaavkyj'])
print(new_detections_df.loc['ZTF18aaavkyj'])

classALeRCE                     Ceph
ra                           192.885
dec                          24.1226
period                       1.09572
source                     CRTSnorth
id_source            1123064031941.0
class_source                    ACEP
separation_arcsec            0.29111
Name: ZTF18aaavkyj, dtype: object
              fid           mjd     magpsf  sigmapsf
oid                                                 
ZTF18aaavkyj    1  58567.446736  16.525101  0.070550
ZTF18aaavkyj    1  58588.213599  15.957800  0.069776
ZTF18aaavkyj    2  58589.241782  15.788184  0.034676


In [10]:
import pandas as pd

### save files
extra_name = '' if uses_corr else '_noncorr'
save_root_dir = f'../../data/{survey_name}'
new_labels_df.to_parquet(f'{save_root_dir}/labels_vs{extra_name}.parquet')
new_detections_df.to_parquet(f'{save_root_dir}/detections_vs{extra_name}.parquet')